In [1]:
cd /home/jovyan/Documents/dsi_plus/Projects/project_3 - Madelon_feature_selection

/home/jovyan/Documents/dsi_plus/Projects/project_3 - Madelon_feature_selection


In [2]:
#Import standard modules
%run lib/__init__.py
%matplotlib inline

### Project 3: Madelon Dataset - Feature Selection + Classification

## Step 2. Feature Selection - SFM (SelectFromModel)

Hyperparameters:
* Esimator = LassoCV - Lasso linear model with iterative fitting along a regularization path
      * include a range of values for c in gridsearch. Small c = high regularisation.
* Threshold = A string, float, optional default None. The threshold value to use for feature selection. 
      * include a range of values for the threshold in the gridsearch. 
      * A low threshold = highly regularised model => high penalty for betas as we decrease regularisation strenght (c) threshold increases as lower penalty on betas. If the threshold is too low the model will not beable to find coeffs and will stop.

Use a StratifiedShuffleSplit from the gridsearch crossfolds, this ensures that relative class frequencies are approximately preserved in each train and validation fold.

In [3]:
random.seed(42)

In [4]:
#Load files names for datasets
pickle_files=Pickle_files()
datafiles=pickle_files.read_datafiles()

In [190]:
#List of classifiers to use for scoring selection
Classifiers={'SFM_KNN':KNeighborsClassifier(),'SFM_LR': LogisticRegression(C=100),\
             'SFM_DT': DecisionTreeClassifier(), 'SFM_SVC':SVC()}
XClassifiers={'SFM_LR': LogisticRegression(C=100, random_state=42)}

In [170]:
def sfm_pipeline(ds,Classifiers, paramG,datafiles=datafiles ):
    ''' 
    Pipline to find best features using SelectFromModel with LogisticRegression estimator
    '''
    X=pd.read_pickle(datafiles[ds][2])
    y=pd.read_pickle(datafiles[ds][4])
    print(datafiles[ds][2],X.shape, datafiles[ds][4],y.shape)
    
    results=[]
    
    for n,model in tqdm(Classifiers.items()):   
       
        
        pipe=Pipeline(steps=[
                                ('scaler' , StandardScaler()),
                                ('sfm' , SelectFromModel(LogisticRegression(penalty='l1',solver='liblinear'))),
                                ('clf' ,model)
                               ])
    
        param=paramG

        grsCV=GridSearchCV(pipe,param,n_jobs=-1, verbose=1,\
                           cv=StratifiedShuffleSplit(n_splits=5, test_size=.2))
       
        grsCV.fit(X,y.squeeze())
        
        
        sfm_support=grsCV.best_estimator_.named_steps['sfm'].get_support()
        
        results_n={ 'dataset' : ds,
                    'model'  : n,
                    'best_test_score' : grsCV.best_score_,
                    'best_params' : grsCV.best_params_ , 
                    'featSelect' : np.where(sfm_support)[0], 
                    'n_featSelect' : np.sum(sfm_support)
                    }
        results.append(results_n)
    
        
    results_df=pd.DataFrame(results)
    
    return results_df

### SFM FOR UCI DATASET

In [171]:
UCI_param=({'sfm__threshold': [.25,.3,.4,.45,.5,.6] ,
            'sfm__estimator': [LogisticRegression(C=c) for c in np.logspace(1,3,3)]})

In [175]:
%%time
UCI_results_df=sfm_pipeline('UCI',Classifiers, UCI_param)

  0%|          | 0/4 [00:00<?, ?it/s]

./data/pickle_data/UCI_X_train (2080, 500) ./data/pickle_data/UCI_y_train (2080, 1)
Fitting 5 folds for each of 18 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    5.1s
[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed:   10.9s finished
 25%|██▌       | 1/4 [00:11<00:34, 11.34s/it]

Fitting 5 folds for each of 18 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    5.6s
[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed:   12.0s finished
 50%|█████     | 2/4 [00:23<00:23, 11.95s/it]

Fitting 5 folds for each of 18 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    5.7s
[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed:   11.4s finished
 75%|███████▌  | 3/4 [00:35<00:11, 11.92s/it]

Fitting 5 folds for each of 18 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    6.4s
[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed:   13.2s finished
100%|██████████| 4/4 [00:49<00:00, 12.38s/it]

CPU times: user 6.99 s, sys: 620 ms, total: 7.61 s
Wall time: 49.6 s


In [176]:
UCI_results_df

,best_params,best_test_score,dataset,featSelect,model,n_featSelect
0,{'sfm__estimator': LogisticRegression(C=1000.0...,0.886058,UCI,"[28, 48, 105, 128, 281, 318, 336, 338, 378, 43...",SFM_KNN,11
1,"{'sfm__estimator': LogisticRegression(C=100.0,...",0.605769,UCI,"[48, 105, 128, 281, 318, 338, 378, 433, 451]",SFM_LR,9
2,"{'sfm__estimator': LogisticRegression(C=100.0,...",0.820192,UCI,"[28, 48, 105, 128, 281, 318, 336, 338, 378, 43...",SFM_DT,11
3,"{'sfm__estimator': LogisticRegression(C=100.0,...",0.862500,UCI,"[28, 48, 105, 128, 281, 318, 336, 338, 378, 43...",SFM_SVC,11


In [178]:
UCI_results_df.best_params[0]

{'sfm__estimator': LogisticRegression(C=1000.0, class_weight=None, dual=False,
           fit_intercept=True, intercept_scaling=1, max_iter=100,
           multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
           solver='liblinear', tol=0.0001, verbose=0, warm_start=False),
 'sfm__threshold': 0.4}

In [180]:
UCI_results_df.to_pickle('./data/pickle_data/UCI_bestfeatures_sfm')

### SFM FOR POSTGRES SAMPLES

#### Sample 1

In [186]:
Sample1_param=({'sfm__threshold': [.25,.4,.5,.6] ,
            'sfm__estimator': [LogisticRegression(C=c) for c in np.logspace(1,3,3)]})

In [187]:
%%time
Sample1_results_df =sfm_pipeline('Sample1',Classifiers, Sample1_param)

  0%|          | 0/4 [00:00<?, ?it/s]

./data/pickle_data/Sample1_X_train (7364, 1000) ./data/pickle_data/Sample1_y_train (7364,)
Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:  4.2min finished
 25%|██▌       | 1/4 [04:19<12:57, 259.20s/it]

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:  4.0min finished
 50%|█████     | 2/4 [08:28<08:28, 254.08s/it]

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:  3.9min finished
 75%|███████▌  | 3/4 [12:34<04:11, 251.43s/it]

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:  4.5min finished
100%|██████████| 4/4 [17:13<00:00, 258.41s/it]

CPU times: user 1min 4s, sys: 1.16 s, total: 1min 5s
Wall time: 17min 13s


In [188]:
Sample1_results_df 

,best_params,best_test_score,dataset,featSelect,model,n_featSelect
0,"{'sfm__estimator': LogisticRegression(C=100.0,...",0.805974,Sample1,"[257, 269, 308, 315, 336, 341, 395, 504, 526, ...",SFM_KNN,20
1,"{'sfm__estimator': LogisticRegression(C=100.0,...",0.679701,Sample1,"[257, 269, 308, 315, 336, 341, 395, 504, 526, ...",SFM_LR,20
2,"{'sfm__estimator': LogisticRegression(C=100.0,...",0.725866,Sample1,"[257, 269, 308, 315, 336, 341, 395, 504, 526, ...",SFM_DT,20
3,"{'sfm__estimator': LogisticRegression(C=100.0,...",0.755872,Sample1,"[257, 269, 308, 315, 336, 341, 395, 504, 526, ...",SFM_SVC,20


In [189]:
Sample1_results_df.best_params[0]

{'sfm__estimator': LogisticRegression(C=100.0, class_weight=None, dual=False, fit_intercept=True,
           intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
           penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
           verbose=0, warm_start=False), 'sfm__threshold': 0.4}

#### Sample 2

In [191]:
%%time
Sample2_results_df =sfm_pipeline('Sample2',Classifiers, Sample1_param)

  0%|          | 0/4 [00:00<?, ?it/s]

./data/pickle_data/Sample2_X_train (7325, 1000) ./data/pickle_data/Sample2_y_train (7325,)
Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:  4.5min finished
 25%|██▌       | 1/4 [04:33<13:40, 273.43s/it]

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:  4.6min finished
 50%|█████     | 2/4 [09:21<09:21, 280.65s/it]

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:  4.7min finished
 75%|███████▌  | 3/4 [14:24<04:48, 288.23s/it]

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:  4.9min finished
100%|██████████| 4/4 [19:33<00:00, 293.48s/it]

CPU times: user 1min 14s, sys: 1.47 s, total: 1min 16s
Wall time: 19min 34s


In [192]:
Sample2_results_df 

,best_params,best_test_score,dataset,featSelect,model,n_featSelect
0,"{'sfm__estimator': LogisticRegression(C=10.0, ...",0.789898,Sample2,"[257, 269, 315, 336, 341, 395, 504, 526, 639, ...",SFM_KNN,18
1,"{'sfm__estimator': LogisticRegression(C=100.0,...",0.683686,Sample2,"[257, 269, 308, 315, 336, 341, 395, 504, 526, ...",SFM_LR,20
2,{'sfm__estimator': LogisticRegression(C=1000.0...,0.726826,Sample2,"[257, 269, 308, 315, 336, 341, 395, 504, 526, ...",SFM_DT,20
3,"{'sfm__estimator': LogisticRegression(C=100.0,...",0.765188,Sample2,"[257, 269, 308, 315, 336, 341, 395, 504, 526, ...",SFM_SVC,20


#### Sample 3

In [193]:
%%time
Sample3_results_df =sfm_pipeline('Sample3',Classifiers,Sample1_param)

  0%|          | 0/4 [00:00<?, ?it/s]

./data/pickle_data/Sample3_X_train (7300, 1000) ./data/pickle_data/Sample3_y_train (7300,)
Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:  4.2min finished
 25%|██▌       | 1/4 [04:24<13:14, 264.96s/it]

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:  4.4min finished
 50%|█████     | 2/4 [09:01<09:01, 270.57s/it]

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:  4.4min finished
 75%|███████▌  | 3/4 [13:37<04:32, 272.67s/it]

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:  4.8min finished
100%|██████████| 4/4 [18:37<00:00, 279.29s/it]

CPU times: user 1min 6s, sys: 1.26 s, total: 1min 7s
Wall time: 18min 37s


In [194]:
Sample3_results_df 

,best_params,best_test_score,dataset,featSelect,model,n_featSelect
0,"{'sfm__estimator': LogisticRegression(C=100.0,...",0.810959,Sample3,"[257, 269, 308, 315, 336, 341, 395, 504, 526, ...",SFM_KNN,20
1,"{'sfm__estimator': LogisticRegression(C=100.0,...",0.684110,Sample3,"[257, 269, 308, 315, 336, 341, 395, 504, 526, ...",SFM_LR,20
2,{'sfm__estimator': LogisticRegression(C=1000.0...,0.737123,Sample3,"[257, 269, 308, 315, 336, 341, 395, 504, 526, ...",SFM_DT,20
3,"{'sfm__estimator': LogisticRegression(C=10.0, ...",0.765616,Sample3,"[257, 269, 308, 315, 336, 341, 395, 504, 526, ...",SFM_SVC,19


In [195]:
Sample_results_df=pd.concat([Sample1_results_df,Sample2_results_df,\
                          Sample3_results_df], axis=0, ignore_index=True)

In [196]:
Sample_results_df.to_pickle('./data/pickle_data/Samples_bestfeatures_sfm')